In [2]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [3]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from datasets import load_dataset

dataset = load_dataset("LeoTungAnh/kdd210_hourly")
dataset

DatasetDict({
    train: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    test: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
})

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [9]:
len(test_dataset[0]['target'])

10898